<a href="https://colab.research.google.com/github/Matheus515/Trabalho_final_IA_genetics/blob/main/Trabalho_Final_IA_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Matheus Otávio Lisboa

##### Bibliotecas/drive

In [9]:
# conexão com o Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


import pandas as pd
import numpy as np

#Scikit-learn
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV,
                                     cross_val_predict, KFold, StratifiedKFold)
from sklearn.metrics import (r2_score, mean_absolute_error, mean_squared_error)
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline

#TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2


import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import gc
import itertools
from joblib import dump
import warnings

warnings.filterwarnings('ignore')
print("Todas as bibliotecas necessárias foram importadas com sucesso.")

Mounted at /content/drive
Todas as bibliotecas necessárias foram importadas com sucesso.


##### Matriz de genes de Proteina e Lactose

In [6]:
#Proteína
geno_p = pd.read_csv('//content//drive//MyDrive//TP_IA_final_108190//geno_chr6_protein_matched.csv')
geno_p

,IID,rs268256475,rs268256476,rs268256478,rs268238334,rs268238335,rs268238337,rs268270001,rs268263985,rs268263939,...,rs268270642,rs268270643,rs268234369,rs268234368,rs268259460,rs268234366,rs268234365,rs268241833,rs268241834,rs268241835
0,850883,2,2,1,2,1,0,0,0,1,...,0,0,0,0,2,2,2,0,0,0
1,850884,1,1,0,1,0,0,0,1,0,...,0,0,0,0,2,2,2,0,0,0
2,850885,0,0,0,1,0,1,0,0,0,...,0,1,1,0,0,1,1,0,2,1
3,850886,1,1,0,1,1,0,0,0,0,...,1,2,0,2,1,1,0,1,1,1
4,850887,1,1,0,2,2,0,0,0,1,...,1,0,1,0,2,1,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,869918,1,1,0,1,1,0,0,1,0,...,1,1,0,0,1,0,0,2,0,2
908,869932,1,1,1,1,2,0,0,1,0,...,1,1,0,0,0,1,1,1,0,2
909,869945,0,0,0,1,1,0,0,1,0,...,0,2,0,1,0,0,1,1,0,1
910,869949,0,0,0,2,2,0,0,0,0,...,2,0,0,0,0,1,1,0,1,0


In [7]:
#Lactose
geno_l = pd.read_csv('//content//drive//MyDrive//TP_IA_final_108190//geno_chr2_lactose_matched.csv')
geno_l

,IID,rs268288171,rs268288170,rs268288169,rs268288168,rs268288167,rs268288165,rs268292999,rs268235814,rs268258298,...,rs268270125,rs268270124,rs268270123,rs268270122,rs268270121,rs268270120,rs268270119,rs268270118,rs268270116,rs268270115
0,850883,1,1,1,1,1,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
1,850884,1,0,1,2,1,1,0,0,0,...,0,1,1,0,1,0,1,0,1,1
2,850885,0,1,1,1,2,1,0,1,0,...,0,2,0,0,2,0,2,0,0,0
3,850886,1,1,1,1,1,1,2,1,1,...,0,2,1,0,1,1,1,1,1,1
4,850887,0,0,0,1,1,0,0,0,0,...,0,1,1,0,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,869918,0,0,1,0,0,0,0,1,0,...,0,1,1,0,1,1,1,0,1,1
908,869932,0,0,0,0,1,1,1,0,0,...,1,0,2,1,0,1,0,2,1,1
909,869945,0,0,0,0,0,1,1,2,0,...,1,2,1,0,2,0,2,0,1,2
910,869949,1,0,2,1,0,2,1,1,1,...,1,2,0,1,1,0,1,1,0,0


##### Tabela de fenótipos por individuo

In [8]:
feno = pd.read_csv('//content//drive//MyDrive//TP_IA_final_108190//pheno_protein_matched.csv')
feno

,ID,Farm,Year_birth,Litter_size,%_Fat,%_Protein,%_Lactose,%_Dry_matter,SCC_1000,MY210,MY240,MY305,Lactation
0,850883,GJE,2015,1,4.42,3.16,4.94,7.58,112.0,487.63,487.63,487.63,1
1,850884,GJE,2015,2,4.75,3.43,4.78,8.18,491.0,496.30,576.83,645.35,1
2,850885,GJE,2015,1,4.59,3.00,4.93,7.60,439.0,442.86,442.86,442.86,1
3,850886,GJE,2016,2,4.99,3.35,4.79,8.35,103.0,355.98,355.98,355.98,1
4,850887,GJE,2016,1,4.47,3.42,4.83,7.89,136.0,431.68,445.90,486.43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,869918,IE,2016,2,4.78,3.70,4.88,8.48,1288.0,301.89,310.43,310.43,1
908,869932,IE,2016,1,5.23,3.46,4.74,8.70,1726.0,449.16,505.91,593.77,1
909,869945,IE,2016,2,5.40,3.47,5.11,8.87,548.0,457.05,494.24,617.68,1
910,869949,IE,2016,2,5.02,3.76,4.80,8.78,1503.0,483.49,534.17,653.39,1


#### RF

In [ ]:
# Configurando entrada dos datasets
PROJECT_PATH = '/content/drive/MyDrive/TP_IA_final_108190/'
print(f"Caminho do projeto definido para: {PROJECT_PATH}")

# Configurando carregamento dados de fenótipo (características proteina/lactose) e de genótipo (marcadores)
# O "map" contem informações sobre os marcadores
file_config = {
    '%_Protein': {
        'pheno': os.path.join(PROJECT_PATH, 'pheno_protein_matched.csv'),
        'geno': os.path.join(PROJECT_PATH, 'geno_chr6_protein_matched.csv'),
        'map': os.path.join(PROJECT_PATH, 'map_chr6_protein.txt')
    },
    '%_Lactose': {
        'pheno': os.path.join(PROJECT_PATH, 'pheno_lactose_matched.csv'),
        'geno': os.path.join(PROJECT_PATH, 'geno_chr2_lactose_matched.csv'),
        'map': os.path.join(PROJECT_PATH, 'map_chr2_lactose.txt')
    }
}
print("Configuração de arquivos por fenótipo foi criada.")

# Loop principal
for phenotype, paths in file_config.items():

    print(f"\n{'='*70}\nIniciando análise para o FENÓTIPO: {phenotype}\n{'='*70}")

    # carregando os dados nos dataframes
    try:
        data = pd.read_csv(paths['pheno'])
        geno = pd.read_csv(paths['geno'], na_values=5)
        map_df = pd.read_csv(paths['map'], delim_whitespace=True).set_index('SNP_ID')
        print(f"Arquivos para '{phenotype}' carregados com sucesso.")
    except FileNotFoundError as e:
        print(f"ERRO: Arquivo não encontrado - {e}. Verifique os nomes e caminhos dos arquivos.")
        continue #pula para o próximo fenótipo

    # pré processamento e otimizando memória
    geno = geno.rename(columns={'IID': 'ID'})
    data = data.set_index('ID')
    geno = geno.set_index('ID')
    common_ids = data.index.intersection(geno.index)
    data = data.loc[common_ids].reset_index()
    geno = geno.loc[common_ids].reset_index()

    # Preenchendo valores nulos com a moda
    print("Otimizando uso de memória da matriz de genótipos...")
    snp_cols = geno.columns[1:]
    geno[snp_cols] = geno[snp_cols].apply(pd.to_numeric, downcast='integer')
    for col in snp_cols:
        if geno[col].isnull().any():
            moda = geno[col].mode()[0]
            geno[col] = geno[col].fillna(moda)
    geno[snp_cols] = geno[snp_cols].astype('int8')
    print("Otimização de memória concluída.")

    X_full = geno.iloc[:, 1:]

    # Rodando nos 2 cenários: com e sem ajuste
    for analysis_type in ['com_ajuste', 'sem_ajuste']:

        # preparando cenario
        scenario_title = (f"FENÓTIPO: {phenotype} | AJUSTE: {analysis_type.split('_')[0].upper()}")
        print(f"\n--- {scenario_title} ---")

        # pastas de saída
        output_dir = os.path.join(PROJECT_PATH, f"rf_{phenotype.replace('%_', '')}_{analysis_type}")
        os.makedirs(output_dir, exist_ok=True)

        # Definindo y e preparando o X
        current_data = data[['ID', 'Farm', 'Year_birth', 'Litter_size', phenotype]].dropna(subset=[phenotype])
        valid_indices = current_data.index
        X_model = X_full.loc[valid_indices].values # Pega os dados genéticos dos animais válidos
        selected_features = X_full.columns # Todos os SNPs são usados

        # Se for com ajuste (faz ajuste), se não (usa valores brutos mesmo)
        if analysis_type == 'com_ajuste':
            print("1. AJUSTANDO FENÓTIPO...")
            formula = f"Q('{phenotype}') ~ C(Farm) + C(Year_birth) + C(Litter_size)"
            model = smf.ols(formula, data=current_data).fit()
            y = model.resid.values
        else:
            print("1. USANDO FENÓTIPO BRUTO...")
            y = current_data[phenotype].values

        print(f"Análise será feita com {len(y)} animais e {X_model.shape[1]} SNPs.")

        # usando paralelismo máximo
        n_jobs_dynamic = -1
        print(f"INFO: Usando n_jobs = {n_jobs_dynamic} para este cenário.")

        # os hiperparametros que serão testados
        print("\n2. CONFIGURANDO PIPELINE (StandardScaler -> RF) E OTIMIZANDO...")
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('rf', RandomForestRegressor(random_state=42))
        ])

        param_grid = {
            'rf__n_estimators': [100, 200, 300], 'rf__max_depth': [20, 30, 40],
            'rf__min_samples_split': [2, 5, 10], 'rf__min_samples_leaf': [1, 2, 4],
            'rf__max_features': ['sqrt', 'log2']
        }

        start_time = time.time()

        # Não testei todas as combinações de hiperparmetros, foram sorteadas algumas
        # Validação cruzada cv = 5
        rf_random = RandomizedSearchCV(
            estimator=pipeline, param_distributions=param_grid, n_iter=10, cv=5,
            verbose=0, random_state=42, n_jobs=n_jobs_dynamic
        )
        rf_random.fit(X_model, y)
        print(f"Otimização concluída em {time.time() - start_time:.2f}s.")

        best_pipeline = rf_random.best_estimator_
        print(f"Melhores parâmetros: {rf_random.best_params_}")

        # Salvando resultados
        model_filename = os.path.join(output_dir, f'rf_pipeline.joblib')
        dump(best_pipeline, model_filename)
        print(f"Pipeline salvo em '{model_filename}'")

        importances = best_pipeline.named_steps['rf'].feature_importances_
        importance_df = pd.DataFrame({"SNP": selected_features, "Importance": importances}) \
                         .sort_values(by="Importance", ascending=False) \
                         .merge(map_df, left_on='SNP', right_index=True, how='left')

        csv_filename = os.path.join(output_dir, f'snp_importances.csv')
        importance_df.to_csv(csv_filename, index=False)
        print(f"Importâncias salvas em '{csv_filename}'")

        # Gerando gráfico e salvando
        print("\n3. GERANDO GRÁFICO dos Top 20 SNPs...")
        top_20 = importance_df.head(20).copy()

        def create_snp_label(row):
            snp_id, chrom = row.get('SNP', ''), row.get('CHR', '')
            if pd.notna(chrom):
                try: return f"Chr {int(chrom)}: {snp_id}"
                except (ValueError, TypeError): return f"Chr {chrom}: {snp_id}"
            return snp_id

        top_20['Label'] = top_20.apply(create_snp_label, axis=1)

        plt.figure(figsize=(12, 10))
        title_text = (f"Top 20 SNPs para '{phenotype}'\n"
                      f"({analysis_type.replace('_', ' ')})")
        plt.title(title_text, fontsize=16)
        sns.barplot(x="Importance", y="Label", data=top_20, palette="viridis_r")
        plt.xlabel("Importância Relativa", fontsize=14)
        plt.ylabel("SNP (Cromossomo: ID)", fontsize=14)
        plt.tight_layout()

        plot_filename = os.path.join(output_dir, f'top_20_snps.png')
        plt.savefig(plot_filename, dpi=300)
        plt.close()
        print(f"Gráfico salvo em '{plot_filename}'")

        # Avaliando modelo com diferentes métricas
        print("\n4. AVALIANDO DESEMPENHO do melhor pipeline...")
        y_pred_cv = cross_val_predict(best_pipeline, X_model, y, cv=5, n_jobs=n_jobs_dynamic)

        r2 = r2_score(y, y_pred_cv)
        mae = mean_absolute_error(y, y_pred_cv)
        rmse = np.sqrt(mean_squared_error(y, y_pred_cv))

        print(f"--- Métricas de Desempenho (R² Score, MAE, RMSE) ---")
        print(f"  {r2:.4f}, {mae:.4f}, {rmse:.4f}")

        #limpeza
        del X_model, y, best_pipeline, importance_df, top_20, y_pred_cv
        gc.collect()

print(f"\n{'='*70}\n ANÁLISE COMPLETA CONCLUÍDA! \n{'='*70}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Bibliotecas importadas e Google Drive montado com sucesso.
Caminho do projeto definido para: /content/drive/MyDrive/TP_IA_final_108190/
Configuração de arquivos por fenótipo foi criada.

Iniciando análise para o FENÓTIPO: %_Protein
Arquivos para '%_Protein' carregados com sucesso.
Otimizando uso de memória da matriz de genótipos...
Otimização de memória concluída.

--- FENÓTIPO: %_Protein | AJUSTE: COM ---
1. AJUSTANDO FENÓTIPO...
Análise será feita com 912 animais e 2357 SNPs.
INFO: Usando n_jobs = -1 para este cenário.

2. CONFIGURANDO PIPELINE (StandardScaler -> RF) E OTIMIZANDO...
Otimização concluída em 71.25s.
Melhores parâmetros: {'rf__n_estimators': 300, 'rf__min_samples_split': 5, 'rf__min_samples_leaf': 4, 'rf__max_features': 'sqrt', 'rf__max_depth': 40}
Pipeline salvo em '/content/drive/MyDrive/TP_IA_final_108190/rf_Protein_com_ajuste/rf_pipeline.j

#### SVM

In [ ]:
PROJECT_PATH = '/content/drive/MyDrive/TP_IA_final_108190/'
print(f"Caminho do projeto definido para: {PROJECT_PATH}")

# Configurando carregamento dados de fenótipo (características proteina/lactose) e de genótipo (marcadores)
# O "map" contem informações sobre os marcadores
file_config = {
    '%_Protein': {
        'pheno': os.path.join(PROJECT_PATH, 'pheno_protein_matched.csv'),
        'geno': os.path.join(PROJECT_PATH, 'geno_chr6_protein_matched.csv'),
        'map': os.path.join(PROJECT_PATH, 'map_chr6_protein.txt')
    },
    '%_Lactose': {
        'pheno': os.path.join(PROJECT_PATH, 'pheno_lactose_matched.csv'),
        'geno': os.path.join(PROJECT_PATH, 'geno_chr2_lactose_matched.csv'),
        'map': os.path.join(PROJECT_PATH, 'map_chr2_lactose.txt')
    }
}
print("Configuração de arquivos por fenótipo foi criada.")

# Loop principal do SVM linear

for phenotype, paths in file_config.items():

    print(f"\n{'='*80}\nIniciando análise SVM LINEAR para o FENÓTIPO: {phenotype}\n{'='*80}")
    # carregando os dados e transformando em dataframe
    try:
        data = pd.read_csv(paths['pheno'])
        geno = pd.read_csv(paths['geno'], na_values=5)
        map_df = pd.read_csv(paths['map'], sep='\s+', engine='python').set_index('SNP_ID')
        print(f"Arquivos para '{phenotype}' carregados com sucesso.")
    except FileNotFoundError as e:
        print(f"ERRO: Arquivo não encontrado - {e}. Verifique os nomes e caminhos dos arquivos.")
        continue

    # pré processando e otimizando memória
    geno = geno.rename(columns={'IID': 'ID'})
    data = data.set_index('ID'); geno = geno.set_index('ID')
    common_ids = data.index.intersection(geno.index)
    data = data.loc[common_ids].reset_index(); geno = geno.loc[common_ids].reset_index()

    print("Otimizando uso de memória e imputando dados ausentes...")
    snp_cols = geno.columns[1:]

    #Verificando valores nulos
    if geno[snp_cols].isnull().values.any():
        print("Valores ausentes detectados. Imputando com a moda...")
        # calcula a moda para todas as colunas de uma vez
        modas = geno[snp_cols].mode().iloc[0]
        # preenche todos os NaNs de uma só vez usando a moda de cada coluna
        geno[snp_cols] = geno[snp_cols].fillna(modas)

    # convertendo para tipo de dado mais eficiente em memória
    geno[snp_cols] = geno[snp_cols].astype('int8')
    print("Otimização de memória e imputação concluídas.")

    X_full = geno.iloc[:, 1:]

    # Fazendo cenários com e sem ajuste
    for analysis_type in ['com_ajuste', 'sem_ajuste']:

        scenario_title = (f"FENÓTIPO: {phenotype} | AJUSTE: {analysis_type.split('_')[0].upper()}")
        print(f"\n--- {scenario_title} ---")

        output_dir = os.path.join(PROJECT_PATH, f"linear_svm_{phenotype.replace('%_', '')}_{analysis_type}")
        os.makedirs(output_dir, exist_ok=True)

        current_data = data[['ID', 'Farm', 'Year_birth', 'Litter_size', phenotype]].dropna(subset=[phenotype])
        valid_indices = current_data.index
        X_model = X_full.loc[valid_indices].values
        selected_features = X_full.columns

        # Se for com ajuste (faz ajuste), se não (usa valores brutos mesmo)
        if analysis_type == 'com_ajuste':
            print("1. AJUSTANDO FENÓTIPO...")
            formula = f"Q('{phenotype}') ~ C(Farm) + C(Year_birth) + C(Litter_size)"
            model = smf.ols(formula, data=current_data).fit()
            y = model.resid.values
        else:
            print("1. USANDO FENÓTIPO BRUTO...")
            y = current_data[phenotype].values

        print(f"Análise será feita com {len(y)} animais e {X_model.shape[1]} SNPs.")

        n_jobs_dynamic = -1
        print(f"INFO: Usando n_jobs = {n_jobs_dynamic} para este cenário.")

        # Verificando hiperparamtros

        print("\n2. CONFIGURANDO PIPELINE (StandardScaler -> LinearSVR) E OTIMIZANDO...")
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('svm', LinearSVR(random_state=42, max_iter=10000, dual="auto"))
        ])
        param_grid = {
            'svm__C': [0.01, 0.1, 1, 10, 100],
            'svm__epsilon': [0, 0.1, 0.2]
        }
        start_time = time.time()

        #validação cruzada cv = 5
        model_search = GridSearchCV(
            estimator=pipeline, param_grid=param_grid, cv=5,
            verbose=1, n_jobs=n_jobs_dynamic
        )
        model_search.fit(X_model, y)
        print(f"Otimização concluída em {time.time() - start_time:.2f}s.")

        best_pipeline = model_search.best_estimator_
        print(f"Melhores parâmetros: {model_search.best_params_}")

        model_filename = os.path.join(output_dir, 'linear_svm_pipeline.joblib')
        dump(best_pipeline, model_filename)
        print(f"Pipeline salvo em '{model_filename}'")

        print("\n3. EXTRAINDO IMPORTÂNCIA DAS FEATURES (COEFICIENTES)...")
        importances = np.abs(best_pipeline.named_steps['svm'].coef_.flatten())
        importance_df = pd.DataFrame({"SNP": selected_features, "Importance": importances}) \
                         .sort_values(by="Importance", ascending=False) \
                         .merge(map_df, left_on='SNP', right_index=True, how='left')

        csv_filename = os.path.join(output_dir, 'snp_importances.csv')
        importance_df.to_csv(csv_filename, index=False)
        print(f"Importâncias (coeficientes) salvas em '{csv_filename}'")

        # criando grafico
        print("\n4. GERANDO GRÁFICO dos Top 20 SNPs...")
        top_20 = importance_df.head(20).copy()

        def create_snp_label(row):
            snp_id, chrom = row.get('SNP', ''), row.get('CHR', '')
            if pd.notna(chrom):
                try: return f"Chr {int(chrom)}: {snp_id}"
                except (ValueError, TypeError): return f"Chr {chrom}: {snp_id}"
            return snp_id
        top_20['Label'] = top_20.apply(create_snp_label, axis=1)

        plt.figure(figsize=(12, 10))
        title_text = (f"Top 20 SNPs (Pesos do Coeficiente) para '{phenotype}'\n"
                      f"Modelo SVM Linear ({analysis_type.replace('_', ' ')})")
        plt.title(title_text, fontsize=16)
        sns.barplot(x="Importance", y="Label", data=top_20, palette="viridis_r")
        plt.xlabel("Importância Absoluta do Coeficiente", fontsize=14)
        plt.ylabel("SNP (Cromossomo: ID)", fontsize=14)
        plt.tight_layout()

        plot_filename = os.path.join(output_dir, 'top_20_snps.png')
        plt.savefig(plot_filename, dpi=300)
        plt.close()
        print(f"Gráfico salvo em '{plot_filename}'")

        print("\n5. AVALIANDO DESEMPENHO do melhor pipeline...")
        y_pred_cv = cross_val_predict(best_pipeline, X_model, y, cv=5, n_jobs=n_jobs_dynamic)

        r2 = r2_score(y, y_pred_cv)
        mae = mean_absolute_error(y, y_pred_cv)
        rmse = np.sqrt(mean_squared_error(y, y_pred_cv))

        print(f"--- Métricas de Desempenho (R² Score, MAE, RMSE) ---")
        print(f"  {r2:.4f}, {mae:.4f}, {rmse:.4f}")

        del X_model, y, best_pipeline, importance_df, top_20, y_pred_cv
        gc.collect()

print(f"\n{'='*80}\n ANÁLISE SVM LINEAR COMPLETA CONCLUÍDA! \n{'='*80}")

Mounted at /content/drive
Bibliotecas importadas e Google Drive montado com sucesso.
Caminho do projeto definido para: /content/drive/MyDrive/TP_IA_final_108190/
Configuração de arquivos por fenótipo foi criada.

Iniciando análise SVM LINEAR para o FENÓTIPO: %_Protein
Arquivos para '%_Protein' carregados com sucesso.
Otimizando uso de memória e imputando dados ausentes...
Valores ausentes detectados. Imputando com a moda...
Otimização de memória e imputação concluídas.

--- FENÓTIPO: %_Protein | AJUSTE: COM ---
1. AJUSTANDO FENÓTIPO...
Análise será feita com 912 animais e 2357 SNPs.
INFO: Usando n_jobs = -1 para este cenário.

2. CONFIGURANDO PIPELINE (StandardScaler -> LinearSVR) E OTIMIZANDO...
Fitting 5 folds for each of 15 candidates, totalling 75 fits
Otimização concluída em 1225.95s.
Melhores parâmetros: {'svm__C': 0.01, 'svm__epsilon': 0.2}
Pipeline salvo em '/content/drive/MyDrive/TP_IA_final_108190/linear_svm_Protein_com_ajuste/linear_svm_pipeline.joblib'

3. EXTRAINDO IMPORTÂ

#### ANN (MLP)

In [ ]:
#Configurando TensorFlow para usar menos memória
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(f"Bibliotecas importadas. Usando TensorFlow v{tf.__version__}")


# Configurando carregamento dados de fenótipo (características proteina/lactose) e de genótipo (marcadores)
# O "map" contem informações sobre os marcadores

PROJECT_PATH = '/content/drive/MyDrive/TP_IA_final_108190/'
file_config = {
    '%_Protein': {
        'pheno': os.path.join(PROJECT_PATH, 'pheno_protein_matched.csv'),
        'geno': os.path.join(PROJECT_PATH, 'geno_chr6_protein_matched.csv'),
        'map': os.path.join(PROJECT_PATH, 'map_chr6_protein.txt')
    },
    '%_Lactose': {
        'pheno': os.path.join(PROJECT_PATH, 'pheno_lactose_matched.csv'),
        'geno': os.path.join(PROJECT_PATH, 'geno_chr2_lactose_matched.csv'),
        'map': os.path.join(PROJECT_PATH, 'map_chr2_lactose.txt')
    }
}
print("Configuração de arquivos criada.")

def create_improved_nn_model(input_dim, learning_rate=0.001, l2_reg=0.01):
    """
    Cria um modelo de rede neural mais robusto com:
    - Arquitetura mais simples para evitar overfitting
    - Regularização L2
    - BatchNormalization
    - Dropout adaptativo
    """
    model = Sequential([
        Input(shape=(input_dim,)),

        # A primeira camada é menor para evitar overfitting
        Dense(32, activation='relu', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(0.2),

        # Segunda camada ainda menor
        Dense(16, activation='relu', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(0.2),

        #camada de saída
        Dense(1, activation='linear')
    ])

    optimizer = Adam(learning_rate=learning_rate, clipnorm=1.0)
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae']
    )
    return model

def create_callbacks():
    """Cria callbacks para early stopping e redução de learning rate"""
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=0
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=8,
        min_lr=1e-6,
        verbose=0
    )

    return [early_stopping, reduce_lr]

print("Função para criar o modelo de Rede Neural melhorado definida.")

# seleciona as top features baseado na correlação absoluta com y
# Ajuda a reduzir overfitting quando há muitas features e melhora tempo de execução do algoritmo
def select_top_features(X, y, n_features=1000):

    if X.shape[1] <= n_features:
        return X, list(range(X.shape[1]))

    correlations = []
    for i in range(X.shape[1]):
        corr = np.corrcoef(X[:, i], y)[0, 1]
        correlations.append(abs(corr) if not np.isnan(corr) else 0)

    top_indices = np.argsort(correlations)[-n_features:]
    return X[:, top_indices], top_indices

# Loop principal
for phenotype, paths in file_config.items():
    print(f"\n{'='*80}\nIniciando análise com REDE NEURAL MELHORADA para o FENÓTIPO: {phenotype}\n{'='*80}")

    # carregando dataframes e pré processando
    try:
        data = pd.read_csv(paths['pheno'])
        geno = pd.read_csv(paths['geno'], na_values=5)
        map_df = pd.read_csv(paths['map'], sep='\s+', engine='python').set_index('SNP_ID')
    except FileNotFoundError as e:
        print(f"ERRO: Arquivo não encontrado - {e}.")
        continue

    geno = geno.rename(columns={'IID': 'ID'})
    data = data.set_index('ID'); geno = geno.set_index('ID')
    common_ids = data.index.intersection(geno.index)
    data = data.loc[common_ids].reset_index(); geno = geno.loc[common_ids].reset_index()
    snp_cols = geno.columns[1:]
    geno[snp_cols] = geno[snp_cols].apply(pd.to_numeric, downcast='integer')

    # Tratando dados nulos (Nan)
    for col in snp_cols:
        if geno[col].isnull().any():
            # substitui valor faltante por moda
            mode_values = geno[col].mode()
            if len(mode_values) > 0:
                moda = mode_values[0]
            else:
                moda = 0  #se nao tiver moda
            geno[col] = geno[col].fillna(moda)

    geno[snp_cols] = geno[snp_cols].astype('int8')
    X_full = geno.iloc[:, 1:]

    for analysis_type in ['com_ajuste', 'sem_ajuste']:
        scenario_title = f"FENÓTIPO: {phenotype} | AJUSTE: {analysis_type.split('_')[0].upper()}"
        print(f"\n--- {scenario_title} ---")

        # define a pasta que vai salvar
        output_dir = os.path.join(PROJECT_PATH, f"nn_melhorado_{phenotype.replace('%_', '')}_{analysis_type}")
        os.makedirs(output_dir, exist_ok=True)

        current_data = data[['ID', 'Farm', 'Year_birth', 'Litter_size', phenotype]].dropna(subset=[phenotype])
        valid_indices = current_data.index
        X = X_full.loc[valid_indices].values
        selected_features_names = X_full.columns

        # Se for com ajuste (faz ajuste), se não (usa valores brutos mesmo)
        if analysis_type == 'com_ajuste':
            print("1. AJUSTANDO FENÓTIPO...")
            formula = f"Q('{phenotype}') ~ C(Farm) + C(Year_birth) + C(Litter_size)"
            model = smf.ols(formula, data=current_data).fit()
            y = model.resid.values
        else:
            print("1. USANDO FENÓTIPO BRUTO...")
            y = current_data[phenotype].values

        print(f"Dados iniciais: {len(y)} animais e {X.shape[1]} SNPs.")

        #seleção de features se houver muitas
        if X.shape[1] > 1000:
            print("2. SELECIONANDO TOP 1000 FEATURES...")
            X, selected_indices = select_top_features(X, y, n_features=1000)
            selected_features_names = selected_features_names[selected_indices]
            print(f"Reduzido para {X.shape[1]} features.")
        else:
            selected_indices = list(range(X.shape[1]))

        #verificando se tem variabilidade suficiente em y
        if np.std(y) < 1e-6:
            print("AVISO: Baixa variabilidade no fenótipo. Pulando análise.")
            continue

        print(f"Análise será feita com {len(y)} animais e {X.shape[1]} SNPs.")

        print("\n3. FASE 1: Otimizando hiperparâmetros com validação cruzada...")

        # grid de hiperparamtros
        param_grid = {
            'epochs': [100],  # Fixo, mas usando usando early stopping
            'batch_size': [16, 32],
            'learning_rate': [0.001, 0.005],
            'l2_reg': [0.001, 0.01]
        }

        param_combinations = list(itertools.product(*param_grid.values()))
        best_score = -np.inf
        best_params = {}

        # Usando KFold simples com 5 folds
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        for i, params in enumerate(param_combinations):
            current_params = dict(zip(param_grid.keys(), params))
            print(f"  Testando {i+1}/{len(param_combinations)}: {current_params}")

            fold_scores = []
            fold_maes = []

            for fold, (train_index, val_index) in enumerate(kf.split(X)):
                X_train, X_val = X[train_index], X[val_index]
                y_train, y_val = y[train_index], y[val_index]

                # Usando RobustScaler pq é mais resistente a outliers
                scaler_X = RobustScaler()
                X_train_scaled = scaler_X.fit_transform(X_train)
                X_val_scaled = scaler_X.transform(X_val)

                # normalizando y também
                scaler_y = StandardScaler()
                y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()

                #Criando o modelo
                model = create_improved_nn_model(
                    input_dim=X_train.shape[1],
                    learning_rate=current_params['learning_rate'],
                    l2_reg=current_params['l2_reg']
                )

                #treinando com early stopping
                callbacks = create_callbacks()

                history = model.fit(
                    X_train_scaled, y_train_scaled,
                    epochs=current_params['epochs'],
                    batch_size=current_params['batch_size'],
                    validation_data=(X_val_scaled, scaler_y.transform(y_val.reshape(-1, 1)).flatten()),
                    callbacks=callbacks,
                    verbose=0
                )

                # Predizendo e "desnormalizando"
                y_pred_scaled = model.predict(X_val_scaled, verbose=0)
                y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

                #calculando as métricas
                r2 = r2_score(y_val, y_pred)
                mae = mean_absolute_error(y_val, y_pred)

                fold_scores.append(r2)
                fold_maes.append(mae)

                #limpando memória
                del model
                tf.keras.backend.clear_session()
                gc.collect()

            avg_r2 = np.mean(fold_scores)
            std_r2 = np.std(fold_scores)
            avg_mae = np.mean(fold_maes)

            print(f"    R² médio: {avg_r2:.4f} ± {std_r2:.4f}, MAE médio: {avg_mae:.4f}")

            if avg_r2 > best_score:
                best_score = avg_r2
                best_params = current_params
                print(f"    *** Novo melhor resultado! ***")

        print(f"\nMelhores parâmetros: {best_params}")
        print(f"Melhor R² médio: {best_score:.4f}")


        print("\n4. FASE 2: Treinamento do modelo final...")

        # preparando dados para o treinamento final
        scaler_X_final = RobustScaler()
        X_scaled_final = scaler_X_final.fit_transform(X)

        scaler_y_final = StandardScaler()
        y_scaled_final = scaler_y_final.fit_transform(y.reshape(-1, 1)).flatten()

        #modelo final
        final_model = create_improved_nn_model(
            input_dim=X.shape[1],
            learning_rate=best_params['learning_rate'],
            l2_reg=best_params['l2_reg']
        )

        # treinamento do modelo final
        callbacks_final = create_callbacks()

        history_final = final_model.fit(
            X_scaled_final, y_scaled_final,
            epochs=150,  # mais épocas para o modelo final
            batch_size=best_params['batch_size'],
            validation_split=0.2,
            callbacks=callbacks_final,
            verbose=1
        )

        # Avaliando o modelo final
        y_pred_final_scaled = final_model.predict(X_scaled_final, verbose=0)
        y_pred_final = scaler_y_final.inverse_transform(y_pred_final_scaled.reshape(-1, 1)).flatten()

        final_r2 = r2_score(y, y_pred_final)
        final_mae = mean_absolute_error(y, y_pred_final)
        final_rmse = np.sqrt(mean_squared_error(y, y_pred_final))

        print(f"\nMétricas do modelo final:")
        print(f"R²: {final_r2:.4f}")
        print(f"MAE: {final_mae:.4f}")
        print(f"RMSE: {final_rmse:.4f}")

        print("\n5. FASE 3: Analisando importância das features...")

        # extraindo os pesos da primeira camada para identificar melhors SNPs
        first_layer_weights = final_model.layers[0].get_weights()[0]
        importances = np.mean(np.abs(first_layer_weights), axis=1)

        #DataFrame de importâncias
        importance_df = pd.DataFrame({
            "SNP": selected_features_names[selected_indices] if len(selected_indices) < len(selected_features_names) else selected_features_names,
            "Importance_Weight": importances
        }).sort_values(by="Importance_Weight", ascending=False)

        #Merge com informações do arquivo map
        importance_df = importance_df.merge(map_df, left_on='SNP', right_index=True, how='left')

        #salvando
        csv_filename = os.path.join(output_dir, f'nn_melhorado_snp_importances.csv')
        importance_df.to_csv(csv_filename, index=False)
        print(f"Importâncias salvas em '{csv_filename}'")


        metrics_df = pd.DataFrame({
            'Metric': ['R2', 'MAE', 'RMSE', 'Best_CV_R2'],
            'Value': [final_r2, final_mae, final_rmse, best_score]
        })
        metrics_filename = os.path.join(output_dir, f'nn_melhorado_metrics.csv')
        metrics_df.to_csv(metrics_filename, index=False)

        #gráfico
        top_20 = importance_df.head(20).copy()

        def create_snp_label(row):
            snp_id = row.get('SNP', '')
            chrom = row.get('CHR', '')
            if pd.notna(chrom):
                try:
                    return f"Chr {int(chrom)}: {snp_id}"
                except (ValueError, TypeError):
                    return f"Chr {chrom}: {snp_id}"
            return snp_id

        top_20['Label'] = top_20.apply(create_snp_label, axis=1)

        plt.figure(figsize=(12, 10))
        title_text = f"Top 20 SNPs - Modelo NN Melhorado\n'{phenotype}' ({analysis_type.replace('_', ' ')}) - R²={final_r2:.3f}"
        plt.title(title_text, fontsize=16)
        sns.barplot(x="Importance_Weight", y="Label", data=top_20, palette="viridis_r")
        plt.xlabel("Importância Média dos Pesos Absolutos", fontsize=14)
        plt.ylabel("SNP (Cromossomo: ID)", fontsize=14)
        plt.tight_layout()

        plot_filename = os.path.join(output_dir, f'nn_melhorado_top_20_snps.png')
        plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Gráfico salvo em '{plot_filename}'")

        # Plotar histórico de treinamento
        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        plt.plot(history_final.history['loss'], label='Treino')
        plt.plot(history_final.history['val_loss'], label='Validação')
        plt.title('Loss durante o treinamento')
        plt.xlabel('Época')
        plt.ylabel('MSE Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history_final.history['mae'], label='Treino')
        plt.plot(history_final.history['val_mae'], label='Validação')
        plt.title('MAE durante o treinamento')
        plt.xlabel('Época')
        plt.ylabel('MAE')
        plt.legend()

        plt.tight_layout()
        history_filename = os.path.join(output_dir, f'nn_melhorado_training_history.png')
        plt.savefig(history_filename, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Histórico de treinamento salvo em '{history_filename}'")

        #Limpando memória
        del final_model, X, y, importance_df, top_20
        tf.keras.backend.clear_session()
        gc.collect()

print(f"\n{'='*80}\n ANÁLISE COM REDE NEURAL MELHORADA COMPLETA! \n{'='*80}")
print("Principais melhorias implementadas:")
print("- Arquitetura mais simples (32-16-1) para evitar overfitting")
print("- Regularização L2 e BatchNormalization")
print("- Early stopping e redução automática de learning rate")
print("- RobustScaler para dados com outliers")
print("- Validação cruzada com 5 folds")
print("- Seleção automática de top features quando há muitas")
print("- Monitoramento detalhado de métricas")
print("- Resultados salvos em pastas 'nn_melhorado_*' para não sobrescrever")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Bibliotecas importadas. Usando TensorFlow v2.18.0
Configuração de arquivos criada.
Função para criar o modelo de Rede Neural melhorado definida.

Iniciando análise com REDE NEURAL MELHORADA para o FENÓTIPO: %_Protein

--- FENÓTIPO: %_Protein | AJUSTE: COM ---
1. AJUSTANDO FENÓTIPO...
Dados iniciais: 912 animais e 2357 SNPs.
2. SELECIONANDO TOP 1000 FEATURES...
Reduzido para 1000 features.
Análise será feita com 912 animais e 1000 SNPs.

3. FASE 1: Otimizando hiperparâmetros com validação cruzada...
  Testando 1/8: {'epochs': 100, 'batch_size': 16, 'learning_rate': 0.001, 'l2_reg': 0.001}
    R² médio: 0.1151 ± 0.0515, MAE médio: 0.2306
    *** Novo melhor resultado! ***
  Testando 2/8: {'epochs': 100, 'batch_size': 16, 'learning_rate': 0.001, 'l2_reg': 0.01}
    R² médio: 0.0601 ± 0.0612, MAE médio: 0.2397
  Testando 3/8: {'epochs': 100, 'batch_size': 16, 'le

## Estrutura de código que foi usada para comparar resultados

Coloque os caminhos adequados dos resultados para testar

Verifica se os top 20 SNPS (features que mais influenciam no fenótipo) encontrados pelo modelo estão na lista de SNPS encontradas usando métodos tradicionais. Ou se possuem correlação acima de 0.3 com algum SNP da lista.

In [ ]:
import pandas as pd
import numpy as np


# Caminho para o resultado (snp_importance) do modelo ue voce quer verificar (Onde pegaremos o top 20 snps)
caminho_rf = ".../snp_importances.csv"

# Caminho para as proteinas ou a lactose que o artigo achou, para comparar com o top 20 do modelo
#(lembrando que para lactose o artigo detectou 1 SNP só, então o top 20 vai ter no maximo 1 em comum)
# Para proteina o artigo detectou varios (mais de 20) então podemos achar até os 20 em comum
caminho_fdr = '.../X._Protein_FDR_markers.txt'

# insira o caminho para a matriz de correlação entre os snps
caminho_corr = "..."

try:
    df_rf = pd.read_csv(caminho_rf)
    df_fdr = pd.read_csv(caminho_fdr, sep='\s+', engine='python')
    df_corr = pd.read_csv(caminho_corr, index_col=0)

    print("Arquivos carregados com sucesso.")

except FileNotFoundError as e:
    print(f"ERRO: Arquivo não encontrado - {e}. Verifique os caminhos e nomes dos arquivos.")

    raise

top_20_snps = df_rf['SNP'].head(20).tolist()
marcadores_fdr = df_fdr['Marker'].tolist()


snps_encontrados_direto = []
snps_encontrados_corr = {}
snps_a_verificar_corr = []

for snp in top_20_snps:
    if snp in marcadores_fdr:
        snps_encontrados_direto.append(snp)
    else:
        snps_a_verificar_corr.append(snp)

for snp in snps_a_verificar_corr:
    if snp in df_corr.index:
        marcadores_fdr_na_matriz = [m for m in marcadores_fdr if m in df_corr.columns]
        correlacoes = df_corr.loc[snp, marcadores_fdr_na_matriz]

        if (correlacoes >= 0.3).any():
            marcador_correlacionado = correlacoes[correlacoes >= 0.3].index[0]
            snps_encontrados_corr[snp] = marcador_correlacionado

snps_encontrados_total = snps_encontrados_direto + list(snps_encontrados_corr.keys())
snps_nao_encontrados = [snp for snp in top_20_snps if snp not in snps_encontrados_total]
contagem = len(snps_encontrados_total)

print("\n" + "="*60)
print("     RESULTADO DA COMPARAÇÃO (IGUALDADE + CORRELAÇÃO ≥ 0.3)")
print("="*60)

print(f"\nDos 20 SNPs mais importantes, {contagem} foram encontrados (diretamente ou por correlação).\n")

if snps_encontrados_total:
    print("--- SNPs ENCONTRADOS ---")
    if snps_encontrados_direto:
        print("\n  Encontrados Diretamente (iguais):")
        for snp in snps_encontrados_direto:
            print(f"    - {snp}")

    if snps_encontrados_corr:
        print("\n  Encontrados por Correlação (r² >= 0.3):")
        for snp, marcador in snps_encontrados_corr.items():
            print(f"    - {snp} (correlacionado com {marcador})")
else:
    print("--- NENHUM SNP FOI ENCONTRADO ---")

print("\n" + "-"*60 + "\n")

if snps_nao_encontrados:
    print("--- SNPs NÃO ENCONTRADOS ---")
    for snp in snps_nao_encontrados:
        print(f"    - {snp}")
else:
    print("--- TODOS OS 20 SNPs FORAM ENCONTRADOS DE ALGUMA FORMA ---")

print("\n" + "="*60)

Arquivos carregados com sucesso.

     RESULTADO DA COMPARAÇÃO (IGUALDADE + CORRELAÇÃO ≥ 0.3)

Dos 20 SNPs mais importantes, 20 foram encontrados (diretamente ou por correlação).

--- SNPs ENCONTRADOS ---

  Encontrados Diretamente (iguais):
    - rs268290908
    - rs268293117
    - rs268290907
    - rs268268356
    - rs268290929
    - rs268240798
    - rs268268321
    - rs268256891
    - rs268273401
    - rs268268357
    - rs268290891
    - rs268268343
    - rs268260302
    - rs268256855
    - rs268273371
    - rs268256901
    - rs268240764
    - rs268290919
    - rs268273398
    - rs268290897

------------------------------------------------------------

--- TODOS OS 20 SNPs FORAM ENCONTRADOS DE ALGUMA FORMA ---



##### Matriz de correlação entre SNPS (marcadores)

In [ ]:
caminho_corr = ".../correlation_r2_chr6_protein.csv"
corr = pd.read_csv(caminho_rf, index_col=0, header=0)
corr

,rs268256475,rs268256476,rs268256478,rs268238334,rs268238335,rs268238337,rs268270001,rs268263985,rs268263939,rs268263940,...,rs268270642,rs268270643,rs268234369,rs268234368,rs268259460,rs268234366,rs268234365,rs268241833,rs268241834,rs268241835
rs268256475,1.000000,0.945678,0.223581,0.053876,0.022130,0.000080,0.079380,0.240566,9.553917e-06,0.045297,...,0.001474,0.003006,0.000006,5.222899e-04,0.004806,0.002733,0.004667,0.005776,0.000113,0.004174
rs268256476,0.945678,1.000000,0.220593,0.047391,0.022879,0.000704,0.072980,0.221228,2.090487e-07,0.040525,...,0.001164,0.003908,0.000032,2.399127e-04,0.003612,0.001718,0.004703,0.006716,0.000246,0.005099
rs268256478,0.223581,0.220593,1.000000,0.049250,0.000723,0.078099,0.191603,0.078540,4.528338e-03,0.001404,...,0.004022,0.005750,0.001343,6.627630e-03,0.000594,0.000751,0.001811,0.000624,0.000083,0.000148
rs268238334,0.053876,0.047391,0.049250,1.000000,0.130314,0.011527,0.033337,0.000187,1.367132e-03,0.057864,...,0.007282,0.000055,0.006806,1.061264e-07,0.000721,0.001199,0.007938,0.001318,0.009850,0.000516
rs268238335,0.022130,0.022879,0.000723,0.130314,1.000000,0.001183,0.033036,0.085403,3.389181e-03,0.002950,...,0.009336,0.000045,0.000086,1.667025e-03,0.003000,0.000010,0.000558,0.000108,0.004181,0.000640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs268234366,0.002733,0.001718,0.000751,0.001199,0.000010,0.003730,0.000774,0.003315,2.394499e-03,0.008618,...,0.017811,0.002494,0.001188,4.844093e-02,0.156868,1.000000,0.152994,0.313855,0.025669,0.095297
rs268234365,0.004667,0.004703,0.001811,0.007938,0.000558,0.000117,0.000631,0.000602,6.790124e-04,0.003527,...,0.000324,0.006227,0.099578,7.964829e-03,0.037480,0.152994,1.000000,0.352564,0.113148,0.097200
rs268241833,0.005776,0.006716,0.000624,0.001318,0.000108,0.000298,0.000880,0.002005,3.439259e-04,0.008675,...,0.004109,0.009950,0.037479,3.018376e-03,0.027419,0.313855,0.352564,1.000000,0.077894,0.513320
rs268241834,0.000113,0.000246,0.000083,0.009850,0.004181,0.000531,0.000037,0.000346,5.138017e-03,0.001207,...,0.030399,0.005236,0.170840,4.858146e-02,0.004505,0.025669,0.113148,0.077894,1.000000,0.134502
